# Tutorial  IUCAA workshop, Mathura
## Astropy Tutorial-3 (Tables) 

Adapted from tutorial by Axel Donath

In [23]:
%matplotlib inline  
import matplotlib.pyplot as plt

In [1]:
import numpy as np
import astropy

In [11]:
from astropy import units as u

## 3. Tables

Astropy provides the [Table](http://docs.astropy.org/en/stable/api/astropy.io.votable.tree.Table.html) class in order to handle data tables.

### 3.1 Basics

Table objects can be created as shown in the following

In [12]:
from astropy.table import Table

In [13]:
table = Table()
print(table)

<No columns>


We add columns to the table like we would add entries to a dictionary

In [14]:
table['Source_Name'] = ['Crab', 'Sag A*', 'Cas A', 'Vela Junior']
table['GLON'] = [184.5575438, 0, 111.74169477, 266.25914205] * u.deg
table['GLAT'] = [-5.78427369, 0, -2.13544151, -1.21985818] * u.deg
table['Source_Class'] = ['pwn', 'unc', 'snr', 'snr']

By executing the following cell, we get a nicely formatted version of the table printed in the notebook:

In [15]:
table

Source_Name,GLON,GLAT,Source_Class
,deg,deg,
str11,float64,float64,str3
Crab,184.5575438,-5.78427369,pwn
Sag A*,0.0,0.0,unc
Cas A,111.74169477,-2.13544151,snr
Vela Junior,266.25914205,-1.21985818,snr


### 3.2 Accessing rows and columns

We have access to the defined columns. To check which ones are availbe you can use `Table.colnames`:

In [16]:
table.colnames

['Source_Name', 'GLON', 'GLAT', 'Source_Class']

And access individual columns just by their name:

In [17]:
table['GLON']

184.5575438
0.0
111.74169477
266.25914205


And also a subset of columns:

In [18]:
table[['Source_Name', 'GLON']]

Source_Name,GLON
,deg
str11,float64
Crab,184.5575438
Sag A*,0.0
Cas A,111.74169477
Vela Junior,266.25914205


Often, it is handy to get the column data as [astropy.units.Quantity](http://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity) using the `.quantity` property:

In [19]:
table['GLON'].quantity

<Quantity [184.5575438 ,   0.        , 111.74169477, 266.25914205] deg>

Rows can be accessed using numpy indexing:

In [20]:
table[0:2]

Source_Name,GLON,GLAT,Source_Class
,deg,deg,
str11,float64,float64,str3
Crab,184.5575438,-5.78427369,pwn
Sag A*,0.0,0.0,unc


Or by using a boolean numpy array for indexing:

In [21]:
selection = table['Source_Name'] == 'Crab'
table[selection]

Source_Name,GLON,GLAT,Source_Class
,deg,deg,
str11,float64,float64,str3
Crab,184.5575438,-5.78427369,pwn


There is also a more sophisticated indexing scheme, which is explained [here](http://docs.astropy.org/en/stable/table/indexing.html), but not covered in this tutorial.

### 3.3 Reading / Writing tables to disk
Astropy tables can be serialized into many formats. For an overview see [here](http://docs.astropy.org/en/latest/io/unified.html#built-in-table-readers-writers). To write the table in FITS format we can use:

In [22]:
table.write('example.fits', overwrite=True, format='fits')

In [23]:
table.write('example.ecsv', overwrite=True, format='ascii.ecsv')

In [24]:
Table.read('example.fits')

Source_Name,GLON,GLAT,Source_Class
,deg,deg,
bytes11,float64,float64,bytes3
Crab,184.5575438,-5.78427369,pwn
Sag A*,0.0,0.0,unc
Cas A,111.74169477,-2.13544151,snr
Vela Junior,266.25914205,-1.21985818,snr


This works for mutiple keys as well:

Astropy's table object also supports the possiblity to group the rows by a given key column. The groups will be defined by the unique values contained in the column defined as key:

In [28]:
table_grouped = table.group_by("Source_Class")

for group in table_grouped.groups:
    print(group, "\n")

Source_Name     GLON        GLAT    Source_Class
                deg         deg                 
----------- ----------- ----------- ------------
       Crab 184.5575438 -5.78427369          pwn 

Source_Name     GLON         GLAT    Source_Class
                deg          deg                 
----------- ------------ ----------- ------------
      Cas A 111.74169477 -2.13544151          snr
Vela Junior 266.25914205 -1.21985818          snr 

Source_Name GLON GLAT Source_Class
            deg  deg              
----------- ---- ---- ------------
     Sag A*  0.0  0.0          unc 



Where each `group` is again a `Table` object:

In [29]:
type(group)

astropy.table.table.Table

### 3.5. Miscellaneous

These are a few other useful operations when working with Astropy tables.

Sort by key:

In [30]:
table.sort('GLON')

In [31]:
table

Source_Name,GLON,GLAT,Source_Class
,deg,deg,
str11,float64,float64,str3
Sag A*,0.0,0.0,unc
Cas A,111.74169477,-2.13544151,snr
Crab,184.5575438,-5.78427369,pwn
Vela Junior,266.25914205,-1.21985818,snr


Note that `.sort()` is an in place operation on the table, i.e. changes the actual table.

To remove a specific row by index:

In [32]:
table.remove_row(0)

Astropy tables also support row-wise iteration in Python loops:

In [33]:
for row in table:
    print(row['Source_Name'])

Cas A
Crab
Vela Junior


Another useful feature for quickly inspecting the data contained in the table is the `.show_in_browser()` method:

In [34]:
table.show_in_browser(jsviewer=True)